In [28]:
# from translate import Translator
from langdetect import detect
from dateutil import parser
import re
import psycopg2
import pandas as pd

import translators as ts
from os import environ

In [29]:
use_date_str = True

In [30]:
conn = psycopg2.connect(
host = environ['DB_HOST'],
port = environ['DB_PORT'],
user = environ['DB_USER'],
password = environ['DB_PASSWORD'],
database = environ['DB_NAME'])

In [31]:
offset = 7
limit = 500

if use_date_str == False:
    query = r"""
        SELECT id, raw_html, language FROM articles
        WHERE article_type = 'blog'
        AND parsed_date IS NULL
        LIMIT {}
    ;""".format(limit)
else:
    query = r"""
        SELECT id, raw_html, posted_date_str, language FROM articles
        WHERE posted_date_str IS NOT NULL
        AND parsed_date IS NULL
        LIMIT {}
    ;""".format(limit)
body = pd.read_sql_query(query, conn)
body

/var/folders/tg/c_k1syfs2b9gvpkyjkcmdqx00000gn/T/ipykernel_88367/4224385000.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  body = pd.read_sql_query(query, conn)


,id,raw_html,posted_date_str,language
0,9618,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 19-JULY",ky
1,9606,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2022-Ж., 22-FEBRUARY",ky
2,9616,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2021-Ж., 2-DECEMBER",ky
3,1290,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2022-Ж., 3-FEBRUARY",ky
4,1287,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2021-Ж., 25-FEBRUARY",ky
...,...,...,...,...
228,13319,"<html dir=""ltr"" class="" js"" data-once=""media-l...","JUNE 4, 2021",en
229,13323,"<html dir=""ltr"" class="" js"" data-once=""media-l...","JULY 3, 2014",en
230,13321,"<html dir=""ltr"" class="" js"" data-once=""media-l...","JUNE 14, 2019",en
231,13324,"<html dir=""ltr"" class="" js"" data-once=""media-l...","MARCH 16, 2015",en


In [32]:
# reg = r"(?<=>)(.*?)(?=<\/)"
reg = r"<p>(.*?)<\/p>"
reg_date = r'<h6 class="posted-date">(.*?)</h6>'

body['raw'] = body['raw_html'].apply(lambda x: ' '.join(re.findall(reg, x)))

if use_date_str == False:
    body['date'] = body['raw_html'].apply(lambda x: ' '.join(re.findall(reg_date, x)).strip().lower().replace(r'[\s\n]+', ' '))
else:
    body['date'] = body['posted_date_str']

In [33]:
def verifyLanguage (row):
    lang = row['language']
    if lang == 'en':
        try:
            return detect(row['raw'])
        except:
            return lang
    else:
        return lang

In [34]:
def translate (row):
    lang = row['verified_language']
    # translator = Translator(from_lang = lang, to_lang = 'en')
    # return translator.translate(row['date'].strip().upper())
    if lang != 'en':
        try:
            return ts.translate_text(row['date'], from_language = lang, to_language = 'en')
        except:
            return row['date']
    else:
        return row['date']

In [35]:
def parseDate (row):
    try:
        return parser.parse(row['date_translation'])
    except:
        return None

In [36]:
body['verified_language'] = body.apply(verifyLanguage, axis = 1)
body['date_translation'] = body.apply(translate, axis = 1)
body['datetime'] = body.apply(parseDate, axis = 1)
body

,id,raw_html,posted_date_str,language,raw,date,verified_language,date_translation,datetime
0,9618,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 19-JULY",ky,<strong>Бишкек – 19-июль 2023-жыл –</strong> Б...,"2023-Ж., 19-JULY",ky,"2023-J., July 19",None
1,9606,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2022-Ж., 22-FEBRUARY",ky,United Nati...,"2022-Ж., 22-FEBRUARY",ky,"2022-J., 22-FEBRUARY",None
2,9616,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2021-Ж., 2-DECEMBER",ky,Чынаранын столунда предметтердин уникалдуу к...,"2021-Ж., 2-DECEMBER",ky,"2021-J., 2-DECEMBER",None
3,1290,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2022-Ж., 3-FEBRUARY",ky,<i>БУУӨПтүн Кыргызстандагы өкүлчүлүгү коммун...,"2022-Ж., 3-FEBRUARY",ky,"2022-J., 3-FEBRUARY",None
4,1287,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2021-Ж., 25-FEBRUARY",ky,"""ДИЛГИР"" тиркемесинин логотиби . Сүрөт: ""Техн...","2021-Ж., 25-FEBRUARY",ky,"2021-J., 25-FEBRUARY",None
...,...,...,...,...,...,...,...,...,...
228,13319,"<html dir=""ltr"" class="" js"" data-once=""media-l...","JUNE 4, 2021",en,Image: UNDP Uzbekistan The current ecologic...,"JUNE 4, 2021",en,"JUNE 4, 2021",2021-06-04 00:00:00
229,13323,"<html dir=""ltr"" class="" js"" data-once=""media-l...","JULY 3, 2014",en,This publication is a unique resource which pr...,"JULY 3, 2014",en,"JULY 3, 2014",2014-07-03 00:00:00
230,13321,"<html dir=""ltr"" class="" js"" data-once=""media-l...","JUNE 14, 2019",en,"On 3 June, the Steering Committee of the Mul...","JUNE 14, 2019",en,"JUNE 14, 2019",2019-06-14 00:00:00
231,13324,"<html dir=""ltr"" class="" js"" data-once=""media-l...","MARCH 16, 2015",en,"The Publication reviews urbanization trends, p...","MARCH 16, 2015",en,"MARCH 16, 2015",2015-03-16 00:00:00


In [37]:
body[body['datetime'].isna()]

,id,raw_html,posted_date_str,language,raw,date,verified_language,date_translation,datetime
0,9618,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2023-Ж., 19-JULY",ky,<strong>Бишкек – 19-июль 2023-жыл –</strong> Б...,"2023-Ж., 19-JULY",ky,"2023-J., July 19",None
1,9606,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2022-Ж., 22-FEBRUARY",ky,United Nati...,"2022-Ж., 22-FEBRUARY",ky,"2022-J., 22-FEBRUARY",None
2,9616,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2021-Ж., 2-DECEMBER",ky,Чынаранын столунда предметтердин уникалдуу к...,"2021-Ж., 2-DECEMBER",ky,"2021-J., 2-DECEMBER",None
3,1290,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2022-Ж., 3-FEBRUARY",ky,<i>БУУӨПтүн Кыргызстандагы өкүлчүлүгү коммун...,"2022-Ж., 3-FEBRUARY",ky,"2022-J., 3-FEBRUARY",None
4,1287,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2021-Ж., 25-FEBRUARY",ky,"""ДИЛГИР"" тиркемесинин логотиби . Сүрөт: ""Техн...","2021-Ж., 25-FEBRUARY",ky,"2021-J., 25-FEBRUARY",None
5,1288,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2019-Ж., 13-DECEMBER",ky,Заманбап технологиялар заманында интернет-ме...,"2019-Ж., 13-DECEMBER",ky,"2019-J., 13-DECEMBER",None
6,1283,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2020-Ж., 10-APRIL",ky,Постер: БУУӨП Кыргызстан <b><i>“Ачык Инновац...,"2020-Ж., 10-APRIL",ky,"2020-J., 10-APRIL",None
7,1275,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2022-Ж., 15-FEBRUARY",ky,<i>ololoOsh креативдүү борборунун базасында ...,"2022-Ж., 15-FEBRUARY",ky,"2022-J., 15-FEBRUARY",None
8,1289,"<html lang=""ky"" dir=""ltr"" class="" js""><head>\n...","2020-Ж., 17-JULY",ky,Айнура Сагынбаева Ишкер айымдар күнүндө. Сүрө...,"2020-Ж., 17-JULY",ky,"2020-J., 17-JULY",None
9,9610,"<html dir=""ltr"" class="" js"" data-once=""media-l...","2021-Ж., 27-APRIL",ky,"Сүрөт: жеке архивден Лия 27 жашта, ал график...","2021-Ж., 27-APRIL",ky,"2021-J., 27-APRIL",None


In [25]:
# translator = Translator(from_lang = 'es', to_lang = 'en')
# translator.translate(r'21 DE FEBREERO DE 2022')
# ts.translate_text('2 DE MAYO DE 2023', from_language = 'es', to_language = 'en')

In [26]:
cur = conn.cursor()

for i, r in body[body['datetime'].notnull()].iterrows():
    sql = """
        UPDATE articles
        SET parsed_date = %s
        WHERE id = %s
    ;"""
    cur.execute(sql, (r['datetime'], r['id']))

conn.commit()
cur.close()

In [27]:
## CLOSE THE CONNECTION TO THE DB
conn.close()